In [1]:
import os
for f in os.listdir():
    if f.startswith("Slide") or (f.endswith(".py") or f.endswith(".tokens")):
        os.remove(f)


In [ ]:
!pip install antlr4-python3-runtime

In [ ]:
!curl -O https://www.antlr.org/download/antlr-4.13.1-complete.jar

# Creando el Grammar

In [2]:
grammar = """
grammar Slide;

program: stat* EOF
    ;

stat: expr ';'
    ;


expr: slide
    ;

slide    : 'Slide' ID '{' (text | img)* '}';
text    : 'Text'  '{' ((transform | text_prop) ';')* '}';
img     : 'Image' '{' ((transform | src) ';')* '}';


transform   : position
            | width
            | height
            ;

text_prop   : textCont
            | fontSize
            | fontFam
            | align
            ;
            
src         : 'src=' STR;
bg          : 'bg=' STR;

position    : 'position=' UNIDAD ',' UNIDAD;
rotation    : 'rotation=' INT;
width       : 'width='  UNIDAD;
height      : 'height='  UNIDAD;

textCont    : 'content=' STR;
fontSize    : 'font-size=' INT;
fontFam     : 'font-family=' STR;
align       : 'align=' ALIGN;

ALIGN: 'center'
    |  'left'
    |  'right'
    |  'justified';

UNIDAD: INT+ ('%' | 'px') ;

INT : [0-9]+ ;
ID: [a-zA-Z_][a-zA-Z_0-9]* ;
STR : '"' (~[\\r\\n])* '"' ;
TEXT_BLOCK : '"""' .*? '"""' -> type(STR);
LINE_COMMENT: '//' ~[\\r\\n]* -> skip;
BLOCK_COMMENT: '/*' .*? '*/' -> skip;

WS  : [ \\t\\r\\n]+ -> skip;
"""
with open("Slide.g4", "w") as f:
    f.write(grammar)

test = """
    Slide Slide1 {
        Text {
            content= \"\"\" esta         es     
            unaa .        prueba
            \"\"\";
            position= 10px, 10px;
            font-size= 30;
            font-family= "Arial";
            align= center;
        }
        Image {
            position= 50%, 50%;
            width= 300px;
            height= 200px;
            src= "image.png";
        }
    };
    Slide Slide2 {
        Image {
            position= 50%, 50%;
            width= 300px;
            height= 200px;
            src= "image.png";
        }
    };
"""

In [3]:
!java -jar antlr-4.13.1-complete.jar -Dlanguage=Python3 Slide.g4

# Probando

In [5]:
from antlr4 import *
from SlideLexer import SlideLexer
from SlideParser import SlideParser

def parse_expression(input_text):
    input_stream = InputStream(input_text)
    lexer = SlideLexer(input_stream)
    token_stream = CommonTokenStream(lexer)
    parser = SlideParser(token_stream)
    tree = parser.prog()
    return tree.toStringTree(recog=parser)

# Ejemplo:
print(parse_expression(test))


(prog (stat (expr (slide Slide Slide1 { (text Text { (text_prop (textCont content= """ esta         es     \n            unaa .        prueba\n            """ ;)) (transform (position position= 10px , 10px ;)) (text_prop (fontSize font-size= 30 ;)) (text_prop (fontFam font-family= "Arial" ;)) (text_prop (align align= center ;)) }) (img Image { (transform (position position= 50% , 50% ;)) (transform (width width= 300px ;)) (transform (height height= 200px ;)) (src src= "image.png" ;) }) })) ;) (stat (expr (slide Slide Slide2 { (img Image { (transform (position position= 50% , 50% ;)) (transform (width width= 300px ;)) (transform (height height= 200px ;)) (src src= "image.png" ;) }) })) ;) <EOF>)
